**요약**
- 4개의 모델이 예측한 logits 또는 submission file을 불러와 평균 앙상블을 수행합니다.

<br>

**Inputs:**
- `dir_data`: 데이터가 있는 디렉토리
- `dir_save`: 제출 파일이 저장될 디렉토리
- `path_cand1`: SegFormer + Pseudo labeling이 예측한 logits들이 저장된 디렉토리
- `path_cand2`: SegFormer + DANN이 예측한 logits들이 저장된 디렉토리
- `path_cand3`: SegFormer + Dice loss가 예측한 logits들이 저장된 디렉토리
- `path_cand4`: Mask2Former가 예측한 submission 파일이 저장된 디렉토리
- `outside_fname`: 공통 배경 마스크가 저장된 피클 파일 이름

<br>

**Outputs**:
- f`{dir_save}/AverageEnsemble_Raw.csv: 평균 앙상블 예측 제출 파일

In [1]:
dir_data = '../data'
dir_save = '../outputs'

path_cand1 = '../outputs/logits_tensors'
path_cand2 = '../outputs/SegFormer'
path_cand3 = '../outputs/SegFormer_Dice'
path_cand4 = '../outputs/Mask2Former/Mask2Former.csv'

outside_fname = '../outputs/total_sum_bg.pickle'

In [2]:
import sys
sys.path.append('../')

import os

import cv2
import torch
import pickle
import numpy as np
import pandas as pd

from tqdm import tqdm

from segformers.utils import rle_encode, rle2mask
from segformers.losses import label_to_one_hot_label

/home/dongjin/miniconda3/envs/torch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv(os.path.join(dir_data, 'test.csv'))

with open(outside_fname, 'rb') as f:
    outside2 = pickle.load(f)
outside2 = cv2.resize(outside2, (960, 540), interpolation=cv2.INTER_NEAREST)


In [4]:
cand4_csv = pd.read_csv(path_cand4)

row = 0
masks = []
result = []

for i in tqdm(range(len(df))):
    cand1 = torch.load(os.path.join(path_cand1, f'prediction_{i}.pt'), map_location='cpu')
    cand1 = torch.softmax(cand1, dim=1)[0].cpu().numpy()

    cand2 = np.load(os.path.join(path_cand2, f'{i}.npy'))[np.newaxis]
    cand2 = torch.as_tensor(cand2)
    cand2 = torch.softmax(cand2, dim=1)[0].cpu().numpy()
    outside1 = np.argmax(cand2, axis=0) == 12

    cand3 = np.load(os.path.join(path_cand3, f'{i}.npy'))[np.newaxis]
    cand3 = torch.as_tensor(cand3)
    cand3 = torch.softmax(cand3, dim=1)[0].cpu().numpy()

    cand4_mask = np.zeros((540, 960)) + 12
    for c in range(12):
        if not cand4_csv.loc[row, 'mask_rle'] == '-1':
            m = rle2mask(cand4_csv.loc[row, 'mask_rle'], label=1)
            cand4_mask[np.where(m == 1)] = c
        row +=1
    cand4 = label_to_one_hot_label(torch.as_tensor(cand4_mask).unsqueeze(0).long(), 13).float() * 0.90
    cand4 = cand4.cpu().numpy()[0]

    logit = (cand1 + cand2 + cand3 + cand4) / 4.0
    mask = logit.argmax(axis=0).astype(np.int32)
    
    mask[np.where(outside1 == 1)] = 12
    mask[np.where(outside2 == 1)] = 12

    # class 0 ~ 11에 해당하는 경우에 마스크 형성 / 12(배경)는 제외하고 진행
    for class_id in range(12):
        class_mask = (mask == class_id).astype(np.int32)
        if np.sum(class_mask) > 0: # 마스크가 존재하는 경우 encode
            mask_rle = rle_encode(class_mask)
            result.append(mask_rle)
        else: # 마스크가 존재하지 않는 경우 -1
            result.append(-1)
            
    masks.append(mask)

100%|██████████| 1898/1898 [09:50<00:00,  3.21it/s]


In [5]:
submit = pd.read_csv(os.path.join(dir_data, 'sample_submission.csv'))
submit['mask_rle'] = result
submit

,id,mask_rle
0,TEST_0000_class_0,212628 5 212636 5 213573 1 213585 1 213587 24 ...
1,TEST_0000_class_1,-1
2,TEST_0000_class_2,598 276 1558 276 2517 278 3476 279 4436 280 53...
3,TEST_0000_class_3,207757 2 208707 21 209656 53 210608 74 210696 ...
4,TEST_0000_class_4,-1
...,...,...
22771,TEST_1897_class_7,152250 8 153208 28 154166 35 155124 38 156083 ...
22772,TEST_1897_class_8,103 532 677 127 857 5 1062 533 1637 127 1817 5...
22773,TEST_1897_class_9,-1
22774,TEST_1897_class_10,-1


In [6]:
submit.to_csv(os.path.join(dir_save, './AverageEnsemble_Raw.csv'), index=False)